In [6]:
import os
import time
os.environ['LSST_DDS_DOMAIN'] = 'auxtelpath'
from salpytools import salpylib 
import logging
logging.basicConfig(level=logging.DEBUG,
                    format='[%(asctime)s] [%(levelname)s] [%(threadName)s]: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


In [7]:
def get_path_from_url(url):
    scp_from = url.split('://')[1]
    scp_to = os.path.expanduser('~/data/')
    filename = os.path.join(scp_to, scp_from.split('/')[-1])
    return scp_from, scp_to, filename
    
def grab_data_from_scp_string(url):
    scp_from,scp_to,filename = get_path_from_url(url)
    
    if not os.path.exists(scp_to):
        os.makedirs(scp_to)
    cmd = 'scp {} {}'.format(scp_from, scp_to)
    #print(cmd)
    proc = subprocess.run(cmd, shell=True, check=True)
    
    if proc.returncode == 0:
        return filename
    else:
        raise IOError('Could not download data from url: {}'.format(url))

def wait_for_data(fname, largeFileObject):
    print('Wait_for_data: {}'.format(fname))
    lfo = None
    if fname is None:
        for i in range(10):
            if len(largeFileObject.LargeFileObjectAvailable.URL) > 0:
                lfo=grab_data_from_scp_string(largeFileObject.LargeFileObjectAvailable.URL)
                print('Got %s' % lfo)
                return lfo
            else:
                print('Waiting for file...')
                time.sleep(0.5)
    else:
        for i in range(10):
            filename = get_path_from_url(str(largeFileObject.LargeFileObjectAvailable.URL))[-1]
            if fname != filename:
                lfo=grab_data_from_scp_string(largeFileObject.LargeFileObjectAvailable.URL)
                print('Got %s' % lfo)
                return lfo
            else:
                print('Waiting for file...')
                time.sleep(0.5)

    return None
    

In [2]:
sedSpectrometer = salpylib.DDSSubscriberContainer('sedSpectrometer')

[2018-07-23 11:25:14] [DEBUG] [MainThread]: Loading Device: sedSpectrometer
[2018-07-23 11:25:14] [DEBUG] [MainThread]: Loading all topics from sedSpectrometer
[2018-07-23 11:25:14] [DEBUG] [MainThread]: Adding AppliedSettingsMatchStart...
[2018-07-23 11:25:21] [DEBUG] [MainThread]: Event subscriber ready for Device:sedSpectrometer topic:AppliedSettingsMatchStart
[2018-07-23 11:25:21] [DEBUG] [MainThread]: Adding DetailedState...
[2018-07-23 11:25:26] [DEBUG] [MainThread]: Event subscriber ready for Device:sedSpectrometer topic:DetailedState
[2018-07-23 11:25:26] [DEBUG] [MainThread]: Adding ErrorCode...
[2018-07-23 11:25:28] [DEBUG] [MainThread]: Event subscriber ready for Device:sedSpectrometer topic:ErrorCode
[2018-07-23 11:25:28] [DEBUG] [MainThread]: Adding Heartbeat...
[2018-07-23 11:25:28] [DEBUG] [MainThread]: Event subscriber ready for Device:sedSpectrometer topic:Heartbeat
[2018-07-23 11:25:28] [DEBUG] [MainThread]: Adding InternalCommand...
[2018-07-23 11:25:29] [DEBUG] [Mai

In [3]:
print(sedSpectrometer.DetailedState.detailedState)
print(sedSpectrometer.SummaryState.summaryState)

0
0


In [4]:
sender = salpylib.DDSSend("sedSpectrometer")

[2018-07-23 11:25:37] [DEBUG] [MainThread]: Loading Device: sedSpectrometer


In [5]:
cmd_id = sender.send_Command('start', settingsToApply='Default', wait_command=True)
print(cmd_id)

[2018-07-23 11:25:41] [DEBUG] [MainThread]: Updating myData object with kwargs
[2018-07-23 11:25:41] [DEBUG] [MainThread]: settingsToApply = Default
[2018-07-23 11:25:41] [DEBUG] [MainThread]: Issuing command: start


(596516649, -301)


In [6]:
cmd_id = sender.send_Command('enable', wait_command=True)
print(cmd_id)

[2018-07-20 17:31:56] [DEBUG] [MainThread]: Updating myData object with kwargs
[2018-07-20 17:31:56] [DEBUG] [MainThread]: Issuing command: enable


(1189641421, -301)


In [15]:
cmd_id = sender.send_Command('captureSpectImage', imageType='test', 
                                             integrationTime=1, lamp='lamp',wait_command=True)
print(cmd_id)

[2018-07-20 17:24:06] [DEBUG] [MainThread]: Updating myData object with kwargs
[2018-07-20 17:24:06] [DEBUG] [MainThread]: imageType = test
[2018-07-20 17:24:06] [DEBUG] [MainThread]: integrationTime = 1
[2018-07-20 17:24:06] [DEBUG] [MainThread]: lamp = lamp
[2018-07-20 17:24:06] [DEBUG] [MainThread]: Issuing command: captureSpectImage


(596516652, -301)


In [ ]:
print(sedSpectrometer.LargeFileObjectAvailable.URL)

In [13]:
sedSpectrometer_fname=wait_for_data(sedSpectrometer_fname, sedSpectrometer)

(596516651, -301)


In [17]:
cmd_id = sender.send_Command('disable', wait_command=True)
print(cmd_id)

[2018-07-20 17:25:07] [DEBUG] [MainThread]: Updating myData object with kwargs
[2018-07-20 17:25:07] [DEBUG] [MainThread]: Issuing command: disable


(1189641422, -301)
